In [ ]:
import os, sys
import shutil

import socketio
import eventlet
import eventlet.wsgi
from flask import Flask
from io import BytesIO

import matplotlib.pyplot as plt
%matplotlib inline

sio = socketio.Server()
app = Flask(__name__)

twiddle = None
pid = None

In [ ]:
class PID:
    def __init__(self, params):
        self.kp = params[0]
        self.ki = params[1]
        self.kd = params[2]
        self.p_error = 0.0
        self.i_error = 0.0
        self.d_error = 0.0
    def updateError(self, cte):
        self.d_error  = cte - self.p_error
        self.p_error  = cte
        self.i_error += cte
    def totalError(self):
        return -(self.kp * self.p_error)  - (self.ki * self.i_error) - (self.kd * self.d_error)

In [ ]:
class Twiddle:
    def __init__(self, tolerance):
        self.tolerance  = tolerance
        # Initial twiddle params
        self.params     = [0, 0, 0]
        self.delta      = [1, 0.1, 1]
        
        self.prev_cte = None
        self.prev_pid = None
        
        self.cur_param  = 0
        self.operations = ["cur_value", "increment", "decrement", "check_decrement"]
        self.next_operation  = "" # ["cur_value", "increment", "decrement"]
        
    def init_pid(self):
        self.params[self.cur_param] += self.delta[self.cur_param]
        self.prev_pid = PID(self.params)       
        self.prev_cte = sys.float_info.max
        self.next_operation = self.operations[1]
        return self.prev_pid
        
    def get_pid(self, cur_cte):
        print("Twiddle details : ")
        print("prev cte :", self.prev_cte)
        print("cur cte  :", cur_cte)
        print("params   :", self.params)
        print("deltas   :", self.delta)
        
        if sum(self.delta) <= self.tolerance:
            print("Best params : ", self.prev_pid.kp, self.prev_pid.ki, self.prev_pid.kd)
            return self.prev_pid            

        if self.next_operation == self.operations[1]:
            if cur_cte < self.prev_cte:
                self.delta[self.cur_param] *= 1.1
                self.prev_cte = cur_cte
                # correct value for self.cur_param. Take next self.cur_param.
                self.cur_param = (self.cur_param + 1) % len(self.params)
                
                self.params[self.cur_param] += self.delta[self.cur_param]
                
                print(self.next_operation, self.params)
                
                self.prev_pid = PID(self.params)
                self.next_operation = self.operations[1]
                return self.prev_pid
            self.next_operation = self.operations[2]
            
        if self.next_operation == self.operations[2]:
            self.params[self.cur_param] -= 2 * self.delta[self.cur_param]
            
            print(self.next_operation, self.params)
            
            self.prev_pid = PID(self.params)       
            # self.prev_cte = cur_cte
            self.next_operation = self.operations[3]
            return self.prev_pid
        
        if self.next_operation == self.operations[3]:
            if cur_cte < self.prev_cte:
                self.delta[self.cur_param] *= 1.1
                self.prev_cte = cur_cte
                # correct value for self.cur_param. Take next self.cur_param.
                self.cur_param = (self.cur_param + 1) % len(self.params)
                
                self.params[self.cur_param] += self.delta[self.cur_param]
                
                print(self.next_operation, self.params)
                
                self.prev_pid = PID(self.params)
                self.next_operation = self.operations[1]
                return self.prev_pid
            
            self.params[self.cur_param] += self.delta[self.cur_param]
            self.delta[self.cur_param] *= 0.9
            # correct value for self.cur_param. Take next self.cur_param.
            self.cur_param = (self.cur_param + 1) % len(self.params)

            self.params[self.cur_param] += self.delta[self.cur_param]
            
            print(self.next_operation, self.params)
            
            self.prev_pid = PID(self.params)
            self.next_operation = self.operations[1]
            return self.prev_pid

In [ ]:
def run(pid, cte, speed):
    pid.updateError(cte)
    steer = pid.totalError()
    throttle = 0.4
    if speed > 40: # speed 25 is cool
        throttle = -0.5
    return steer, throttle

In [ ]:
def send_data(steer, throttle):
    datum={
            "steering_angle": steer,
            "throttle": throttle
        }
    return datum

In [ ]:
app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)